# Datacamp Data Analyst Associate Practical Exam Submission

In [45]:
library(tidyverse)
library(data.table)

### Reading Data

In [46]:
df <-  read.csv("food_claims_2212.csv",colClasses = c("claim_id" = "character"))
head(df)

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
,<chr>,<int>,<chr>,<dbl>,<chr>,<int>,<lgl>,<chr>
1,0001,317,R$ 74474.55,51231.37,RECIFE,15,FALSE,unknown
2,0002,195,R$ 52137.83,42111.30,FORTALEZA,12,TRUE,unknown
3,0003,183,R$ 24447.2,23986.30,SAO LUIS,10,TRUE,meat
4,0004,186,R$ 29006.28,27942.72,FORTALEZA,11,FALSE,meat
5,0005,138,R$ 19520.6,16251.06,RECIFE,11,FALSE,vegetable
6,0006,183,R$ 47529.14,38011.98,NATAL,11,FALSE,unknown


### Checking DataType and NA Count

In [47]:
df %>% map_df(typeof) %>% gather %>% left_join(
df %>% map_df(~sum(is.na(.))) %>% gather, by = "key") %>% 
select(variable = key, datatype = value.x, na_cnt = value.y)

variable,datatype,na_cnt
<chr>,<chr>,<int>
claim_id,character,0
time_to_close,integer,0
claim_amount,character,0
amount_paid,double,36
location,character,0
individuals_on_claim,integer,0
linked_cases,logical,26
cause,character,0


# Task 1

## ### Task 1 (a) State whether the values match the description given in the table above.

In [48]:
### claim_id
#df <-  read.csv("food_claims_2212.csv",colClasses = c("claim_id" = "character")) #run above
class(df$claim_id) # "character"
n_distinct(df$claim_id) == nrow(df) # TRUE which means the claim_id is unique, without any duplicates
sum(is.na(df$claim_id)) #0 there are no NAs present in claim_id

[1] "character"

[1] TRUE

[1] 0

### claim_id
* `claim_id` : In the csv data, values are present 0001, 0014 etc i.e with 0 prefix. So after checking the file, this column must be read as character using `read.csv("food_claims_2212.csv",colClasses = c("claim_id" = "character"))`
* If colClasses will not be used, this column will be read as integer.
* I also compared claim_id with time_to_close variable, latter does not have prefixed 0's in it which emphaises on the claim_id being character.

In [49]:
#time_to_close
is.integer(df$time_to_close) #TRUE The numbers are integers which is counted as discrete
sum(df$time_to_close > 0) == nrow(df) #TRUE; all +ve as count of positives = row count
sum(is.na(df$time_to_close)) #0 No NA count in time_to close

[1] TRUE

[1] TRUE

[1] 0

### time_to_close
* is integer which can be counted as discrete.
* All the values are positive.
* There are no NA values.

In [50]:
# claim_amount
class(df$claim_amount) #character
# The data type is mentioned as continuous but it is character as prefix `R$` is present
# so, cleaning claim amount column and converting to character
df$claim_amount <- as.numeric(gsub("R\\$ ", "",df$claim_amount))
class(df$claim_amount) #numeric
sum(is.na(df$claim_amount)) #0 No NA values present

[1] "character"

[1] "numeric"

[1] 0

### claim_amount
 * In description, it is continuous but its datatype is character as prefix `R$` is present
 * To convert it into numeric, prefix R$ is removed and then convert it to numeric
 * There are no NA values in the column

In [51]:
#amount_paid
class(df$amount_paid) # numeric
sum(is.na(df$amount_paid)) # 36 NA values
df$amount_paid <- replace_na(df$amount_paid, median(df$amount_paid,na.rm = T)) # Imputing NA by median

[1] "numeric"

[1] 36

### amount_paid
* data type is numeric (continuous) as mentioned in description
* there are 36 NA values
* NA values will be replaed by Median value

In [52]:
# location
class(df$location) #character
sum(is.na(df$location)) #0 

[1] "character"

[1] 0

### location
* Thee description mentioned is Nominal which is similar to character as location has no order.
* There are no NA values to replace

In [53]:
# individuals_on_claim
class(df$individuals_on_claim) # integer
min(df$individuals_on_claim) # 1 
sum(is.na(df$individuals_on_claim))

[1] "integer"

[1] 1

[1] 0

### individuals_on_claim
* class mentioned in description is discrete which is similar to the data type integer
* minimum value is 1
* No NA values

In [54]:
# linked_cases
class(df$linked_cases) #'logical'
table(df$linked_cases,useNA = "ifany") #only TRUE,, FALSE and NA present
df$linked_cases <- replace_na(df$linked_cases, FALSE) #replacing NA with FALSE

[1] "logical"


FALSE  TRUE  <NA> 
 1493   481    26 

### linked_cases
* the description is nominal which is in line with logical and can be considered as character without order.
* there are 36 NAs present 
* NAs are replaced by FALSE

In [55]:
# cause
class(df$cause) #character
sum(is.na(df$cause)) #0

[1] "character"

[1] 0

### cause
* The description is nominal which is in line with the datatype character and no order is present in the column
* there are no NAs

### Task 1 (b) State the number of missing values in the column.
* Using summary checked above, it is evident that

* `amount_paid` has 36 NA present

* `linked_cases` has 26 NA present

* Rest of the columns have 0 NA present

* For code, please refer 3rd code cell.

### Task 1 (c) Describe what you did to make values match the description if they did not match.
* `claim_id` column was not character so while reading the csv file, it was converted to character using colclasses i.e. `colClasses = c("claim_id" = "character")``
* `claim amount` is character due to prefix `R$`, We will remove the prefix `R$` and convert the remaining character into numeric using below code
* NAs in `linked_cases` & `amount_paid` were imputed with 'FALSE' & median value

In [56]:
# claim_id column was not character, so while reading the file it was converted to character
#cleaning claim amount column and converting to character
df$claim_amount <- as.numeric(gsub("R\\$ ", "",df$claim_amount))

#NA imputation for columns having NA
df$linked_cases <- replace_na(df$linked_cases, FALSE)
df$amount_paid <- replace_na(df$amount_paid, median(df$amount_paid,na.rm = T))

# Task 2

![image-2](image-2.png)

The plot shows the frequency of observations for each of the location.

## 2(a) The location *"RECIFE"* has most observations which is evident from the above plot.

In [57]:
# 2(a) location having max observations is found out using below code as well 	
df %>% group_by(location) %>% summarise(obs_cnt = n()) %>% 
  arrange(desc(obs_cnt)) %>% pull(location) %>% .[1]

[1] "RECIFE"

In [58]:
#2(b)
df %>% group_by(location) %>% summarise(obs_cnt = n()) %>% summary(obs_cnt)

   location            obs_cnt   
 Length:4           Min.   :287  
 Class :character   1st Qu.:305  
 Mode  :character   Median :414  
                    Mean   :500  
                    3rd Qu.:609  
                    Max.   :885  

### 2(b) The observations across locations are moderately imbalanced ranging from *287 to 885*. The above cell contains the summary statistics

# Task 3

![image-3](image-3.png)

#### The scatter plot of time to close with count of observations shows the number of observations at each time_to_close. From the graph it is evident that most of the observations are having time_to_close between *75 & 275*


## The plot is right skewed

![image-4](image-4.png)
#### The histogram plot of Time_to_close shows the count of observations for various bands of the time. This Plot will tell the skewness of the distribution

# Task 4

![image-5](image-5.png)


In [59]:
df %>% group_by(location) %>% summarise(minn = min(time_to_close),maxx = max(time_to_close), meann = mean(time_to_close), stddev = sd(time_to_close), range = maxx- minn) 

location,minn,maxx,meann,stddev,range
<chr>,<int>,<int>,<dbl>,<dbl>,<int>
FORTALEZA,76,453,185.3055,47.58300,377
NATAL,93,361,185.9268,47.05916,268
RECIFE,82,427,184.6079,48.01325,345
SAO LUIS,84,518,187.1702,53.12609,434


* From the box plot, it is evident that the mean & Inter quartile Range of the time_to_close for each of the location is similar.

* Though we are observing more outliers in `"SAO LUIS"` but it is not having much effect on the region's statistics.¶

* The mean and standard deviation is mostly similar for all 4 locations except `"SAO LUIS"`

* Min, Max & Range have variations across all the locations